In [1]:
import requests
import pandas as pd
import json

In [2]:
URL="https://www.dlsu.edu.ph"

In [3]:
page = requests.get(URL)

# Extract all sub-urls from dlsu.edu.ph

In [4]:
import requests
import html5lib
from bs4 import BeautifulSoup
import re
import time

soup = BeautifulSoup(page.content, 'lxml')
data = []
links = []

url_limit = 50

start = time.time()

def remove_duplicates(l): # remove duplicates and unURL string
    for item in l:
        match = re.search("(?P<url>https?://[^\s]+)", item)
        if match is not None:
            links.append((match.group("url")))


for link in soup.find_all('a', href=True):
    data.append(str(link.get('href')))
    
flag = True
remove_duplicates(data)

while flag:
    try:
        for link in links:
            for j in soup.find_all('a', href=True):
                temp = []
                source_code = requests.get(link)
                soup = BeautifulSoup(source_code.content, 'lxml')
                temp.append(str(j.get('href')))
                remove_duplicates(temp)

                if len(links) > url_limit: # set limitation to number of URLs
                    break
            if len(links) > url_limit:
                break
        if len(links) > url_limit:
            break
    except Exception as e:
        print(e)
        if len(links) > url_limit:
            break
            
for url in links:
    print(url)
    
print ("Elapsed Time: %s" % (time.time() - start))

https://www.dlsu.edu.ph/about-dlsu/
https://www.dlsu.edu.ph/about-dlsu/ourcorevalues/
https://www.dlsu.edu.ph/offices/qao/accreditation/
https://www.dlsu.edu.ph/offices/office-of-the-chancellor/
https://www.dlsu.edu.ph/university-fellows/
https://www.dlsu.edu.ph/emeriti/
https://www.dlsu.edu.ph/brothers/
https://www.dlsu.edu.ph/lasallianmission/
https://www.dlsu.edu.ph/offices/
https://www.dlsu.edu.ph/inside/board-advisors/
https://www.dlsu.edu.ph/admissions/scholarships/
https://www.dlsu.edu.ph/admissions/graduate/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/
https://www.dlsu.edu.ph/colleges/
https://www.dlsu.edu.ph/colleges/gcoe/overview/
http://www.dlsu.edu.ph/wp-content/uploads/pdf/laguna-campus/dlsu-laguna-campus-viewbook.pdf
https://www.dlsu.edu.ph/admissions/graduate/graduate-viewbooks/
https://www.dlsu.edu.ph/wp-content/uploads/pd

In [5]:
web_index = 10 # for testing purposes
print(links[web_index])

https://www.dlsu.edu.ph/admissions/scholarships/


# Get Email

In [6]:
def deCFEmail(fp):
    try:
        r = int(fp[:2],16)
        email = ''.join([chr(int(fp[i:i+2], 16) ^ r) for i in range(2, len(fp), 2)])
        return email
    except (ValueError):
        pass

In [7]:
emailList = []
r = 0

def emailExtractor(urlString):
    getH=requests.get(urlString)
    h=getH.content
    soup=BeautifulSoup(h,'html.parser')
    mailtos = soup.select('a[href]')
    for i in mailtos:
        href=i['href']
        if "email-protect" in href:
            emailList.append(deCFEmail(href.split("#")[1]))

In [8]:
emailExtractor('https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/')
emailList

finalList = set(emailList)
finalList

{'arlyn.ong@dlsu.edu.ph',
 'briane.samson@dlsu.edu.ph',
 'charibeth.cheng@dlsu.edu.ph',
 'christine.diane.ramos@dlsu.edu.ph',
 'clement.ong@dlsu.edu.ph',
 'edward.tighe@dlsu.edu.ph',
 'ethel.ong@dlsu.edu.ph',
 'gregory.cu@dlsu.edu.ph',
 'lissa.magpantay@dlsu.edu.ph',
 'ma.christine.gendrano@dlsu.edu.ph',
 'ma.rowena.caguiat@dlsu.edu.ph',
 'marnel.peradilla@dlsu.edu.ph',
 'michelle.ching@dlsu.edu.ph',
 'neil.delgallego@dlsu.edu.ph',
 'oliver.malabanan@dlsu.edu.ph',
 'rafael.cabredo@dlsu.edu.ph',
 'ronald.pascual@dlsu.edu.ph',
 'ryan.dimaunahan@dlsu.edu.ph',
 'shirley.chu@dlsu.edu.ph'}

# Get Names

# Threading 

## wala pang ginagawa to

In [12]:
import threading
import time

start = time.time()

def fetch_url(url):
    print ("'%s\' fetched in %ss" % (url, (time.time() - start)))
    
threads = [threading.Thread(target=fetch_url, args=(url,)) for url in links]

for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()

print ("Elapsed Time: %s" % (time.time() - start))

'https://www.dlsu.edu.ph/about-dlsu/' fetched in 0.0020012855529785156s
'https://www.dlsu.edu.ph/about-dlsu/ourcorevalues/' fetched in 0.0031528472900390625s
'https://www.dlsu.edu.ph/offices/qao/accreditation/' fetched in 0.00414729118347168s
'https://www.dlsu.edu.ph/offices/office-of-the-chancellor/' fetched in 0.005147457122802734s
'https://www.dlsu.edu.ph/university-fellows/' fetched in 0.006144523620605469s
'https://www.dlsu.edu.ph/emeriti/' fetched in 0.007144451141357422s
'https://www.dlsu.edu.ph/brothers/' fetched in 0.007144451141357422s
'https://www.dlsu.edu.ph/lasallianmission/' fetched in 0.008144378662109375s
'https://www.dlsu.edu.ph/offices/' fetched in 0.009144783020019531s
'https://www.dlsu.edu.ph/inside/board-advisors/' fetched in 0.010145187377929688s
'https://www.dlsu.edu.ph/admissions/scholarships/' fetched in 0.010145187377929688s
'https://www.dlsu.edu.ph/admissions/graduate/' fetched in 0.011144399642944336s
'https://www.dlsu.edu.ph/integrated-school/' fetched in 0

'https://animospace.dlsu.edu.ph' fetched in 0.14314484596252441s
'http://gateway.dlsu.edu.ph' fetched in 0.1441483497619629s
'https://www.dlsu.edu.ph/bituin/hr/' fetched in 0.14514446258544922s
'https://theconcierge.dlsu.edu.ph/' fetched in 0.1461472511291504s
'https://www.dlsu.edu.ph/give/' fetched in 0.14714503288269043s
'https://www.dlsu.edu.ph/offices/registrar/document_request/' fetched in 0.1481478214263916s
'https://www.dlsu.edu.ph/privacy/' fetched in 0.1521458625793457s
'https://www.dlsu.edu.ph/about-dlsu/' fetched in 0.1541454792022705s
Elapsed Time: 0.15514636039733887


# Get emails protected by cloudflare

# Pwede na ata di isama to kasi ayaw gumana

In [10]:
mydivs = soup.findAll("span",{"class": "__cf_email__"})

length = len(mydivs)

for x in range(length):
    a = cfDecodeEmail(mydivs[x]["data-cfemail"])
    print(a)


Quality@DLSU
Jobs@DLSU
Quality@DLSU
Jobs@DLSU


In [11]:
print (mydivs[1]["data-cfemail"])
print(len(mydivs))

4d07222f3e0d09011e18
4
